In [1]:
!pip install -q transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 16.9 MB/s eta 0:00:00


In [2]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType

In [3]:
import pandas as pd
from datasets import Dataset

In [4]:
dataset = load_dataset("yahma/alpaca-cleaned", split="train")
print(dataset[0])

README.md: 0.00B [00:00, ?B/s]

alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

{'output': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.', 'input': '', 'instruction': 'Give three tips for staying healthy.'}


In [5]:
def format_example(example):
    return {
        "text": f"### Instruction:\n{example['instruction']}\n### Input:\n{example['input']}\n### Response:\n{example['output']}"
    }

dataset = dataset.map(format_example)

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [6]:
print(dataset[0]["text"])

### Instruction:
Give three tips for staying healthy.
### Input:

### Response:
1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.

2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.

3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.


In [7]:
#Load base pretrained (non-instruction fine-tuned)
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [9]:
def tokenize_and_mask(example):
    text = example["text"]
    enc = tokenizer(text, truncation=True, padding="max_length", max_length=512)
    input_ids = enc["input_ids"]

    marker = "### Response:"
    start = text.find(marker)
    if start != -1:
        response_start = len(tokenizer(text[:start + len(marker)])["input_ids"])
    else:
        response_start = 0

    labels = input_ids.copy()
    labels[:response_start] = [-100] * response_start
    enc["labels"] = labels
    return enc

tokenized_dataset = dataset.shuffle(seed=42).select(range(2000)).map(tokenize_and_mask, batched=False)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [10]:
# Load model on GPU with 8-bit to save VRAM
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [11]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
    bias="none"
)

In [12]:
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [13]:
training_args = TrainingArguments(
    output_dir="./tinyllama-instruction",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_total_limit=1,
    report_to="none"
)

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

The model is already on multiple devices. Skipping the move to device specified in `args`.


In [15]:
trainer.train()

Step,Training Loss
10,11.785200
20,4.822900
30,0.770900
40,0.476800
50,0.469200
60,0.421900
70,0.369800
80,0.300000
90,0.371100
100,0.366400


TrainOutput(global_step=250, training_loss=1.0377084093093871, metrics={'train_runtime': 390.7504, 'train_samples_per_second': 5.118, 'train_steps_per_second': 0.64, 'total_flos': 6362964688896000.0, 'train_loss': 1.0377084093093871, 'epoch': 1.0})

In [16]:
save_path = "/content/tinyllama-instruction"
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

('/content/tinyllama-instruction/tokenizer_config.json',
 '/content/tinyllama-instruction/special_tokens_map.json',
 '/content/tinyllama-instruction/chat_template.jinja',
 '/content/tinyllama-instruction/tokenizer.model',
 '/content/tinyllama-instruction/added_tokens.json',
 '/content/tinyllama-instruction/tokenizer.json')

In [17]:
instruction_model = AutoModelForCausalLM.from_pretrained(save_path, device_map="auto")

In [18]:
questions = [
    "Explain the mechanism of action of Metformin.",
    "List two advantages of combining Atorvastatin with Ezetimibe.",
    "Summarize how mRNA vaccines work and mention one current research focus."
]

for q in questions:
    print(f"\n🔹 Question: {q}\n")

    prompt = f"### Instruction:\n{q}\n### Response:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = instruction_model.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        repetition_penalty=1.1
    )

    print("✅ Instruction-tuned Output:\n")
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    print("=" * 80)


🔹 Question: Explain the mechanism of action of Metformin.

✅ Instruction-tuned Output:

### Instruction:
Explain the mechanism of action of Metformin.
### Response:
Metformin is an oral antidiabetic drug that acts by inhibiting the synthesis and secretion of glucose by insulin-producing cells, specifically pancreatic β-cells. This inhibition results in a decrease in glucose uptake and storage in tissues, leading to a decrease in blood glucose levels. In addition, it also stimulates insulin secretion from beta-cells, which further enhances the glucose-lowering effects of metformin.
The mechanism of action of metformin involves several steps. Firstly, it binds to glucose transporters on the cell membrane, causing an increase

🔹 Question: List two advantages of combining Atorvastatin with Ezetimibe.

✅ Instruction-tuned Output:

### Instruction:
List two advantages of combining Atorvastatin with Ezetimibe.
### Response:
Combining Atorvastatin with Ezetimibe is a common approach for reduc